# Feature engineering for orders model
This notebook builds most features for the orders model (except for w2vec features that are calculated in a separate notebook).
It takes as input already generated candidates, co-visitation matrixes and some pre-calculations made aside in create_counts_for_buys notebook. Most of the features are built using functions common for all the three models and defined in otto_common_fe notebook, but a few features unique to orders model are built using functions defined in this notebook.

## Imports and definitions

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gc
from datetime import datetime
from humanize import naturalsize

# functions and classes common for several notebooks of current project
import otto_common 
import otto_common_fe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 47.5 MB/s eta 0:00:00


In [2]:
# Polars library is used to generate features from co-visitation matrix.
!pip install polars
import polars as pl

In [3]:
# This function was used for data exploration only.
# It selects ground truth aids that were missed during candidate generation.
def cand_read_and_reduce_test_not_cands(cand_path, is_cv):
    df = pd.read_parquet(cand_path)
    df = df.loc[df['pred_true'] == 0]
    df = df[['session', 'orders']]
    df = df.explode('orders').reset_index(drop=True)
    df['session'] = df['session'].astype(np.int32)
    df['order_predictions'] = df['orders'].astype(np.int32)
    del df['orders']
    gc.collect()
    return df

In [4]:
# Number of buys per aid in history (full sessions).
def add_total_buys(train_path, df_candidate):
    total_counts = pd.read_parquet(train_path)
    total_counts = total_counts.loc[total_counts['type'] > 0]
    total_counts = total_counts.groupby('aid')['session'].nunique()
    gc.collect()
    total_counts.name = "aid_counts_buys"
    df_candidate = pd.merge(df_candidate, total_counts, how='left',
                            left_on='order_predictions', right_index=True)
    df_candidate['aid_counts_buys'] = df_candidate['aid_counts_buys'].fillna(0).astype(np.int16)
    return df_candidate

In [5]:
# Number of orders per aid in history (full sessions).
def add_total_orders(train_path, df_candidate):
    total_counts = pd.read_parquet(train_path)
    total_counts = total_counts.loc[total_counts['type'] == 2]
    total_counts = total_counts.groupby('aid')['session'].nunique()
    gc.collect()
    total_counts.name = "aid_counts_orders"
    df_candidate = pd.merge(df_candidate, total_counts, how='left',
                            left_on='order_predictions', right_index=True)
    df_candidate['aid_counts_orders'] = df_candidate['aid_counts_orders'].fillna(0).astype(np.int16)
    return df_candidate

In [6]:
# Types of buys in session (2 - if at least one aid ordered, 1 - if no orders in history, but at least one aid was carted, 0 if only clicks in session).
def add_buys_in_session(sessions_path, df_candidate):
    df_session = pd.read_parquet(sessions_path)
    df_session = df_session.groupby('session').agg({'type':'max'})
    df_session = df_session.rename(columns={'type':'buys_in_session'})
    df_candidate = pd.merge(df_candidate, df_session, how='left', left_on='session', right_index=True)
    return df_candidate

In [7]:
# Adds a feature showing how often aid was ordered in case if it was clicked, carted or ordered before.
def join_the_2order_conv(all_2order_conv_path, df_candidate):
    df_conv = pd.read_parquet(all_2order_conv_path)
    df_candidate = pd.merge(df_candidate, df_conv, how='left', left_on='order_predictions', right_on='aid')
    df_candidate['total_2order_conv'] = 0
    df_candidate.loc[df_candidate['type_last'] == 0, 'total_2order_conv'] = df_candidate['click2order_conv']
    df_candidate.loc[df_candidate['type_last'] == 1, 'total_2order_conv'] = df_candidate['cart2order_conv']
    df_candidate.loc[df_candidate['type_last'] == 2, 'total_2order_conv'] = df_candidate['order2order_conv']
    df_candidate['total_2order_conv'] = df_candidate['total_2order_conv'].astype(np.float32)
    del df_candidate['aid'], df_candidate['click2order_conv'], df_candidate['cart2order_conv']
    del df_candidate['order2order_conv']
    return df_candidate

In [8]:
# Generates a feature using buy2buy co-visitation matrix.
# Uses all the orders in session and only recent carts. For sessions with no orders and no recent carts, the feature is always 0.
def add_buy2buy_weight(input_path, buy2buy_matrice, df_candidate):
    n_max = 40
    col_name = 'wgt_buy2buy'
    df_test = pd.read_parquet(input_path)
    df_test['time_delta'] = df_test.groupby('session')['ts'].transform(np.max).astype(np.int32)
    df_test['time_delta'] = df_test['time_delta'] - df_test['ts']
    df_test = df_test.loc[((df_test['time_delta'] < 24 * 60 * 60) & (df_test['type'] == 1)) | (df_test['type'] == 2)]
    gc.collect()
    df_test = df_test.sort_values(['session','ts'],ascending=[True,False])
    df_test['n'] = df_test.groupby('session').cumcount().astype(np.int16)
    df_test = df_test.loc[df_test['n'] < n_max]
    df_test = df_test.drop('time_delta', axis=1)
    df_test = df_test[['session', 'aid', 'n']]
    gc.collect()
    
    df_candidate = pl.from_pandas(df_candidate)
    df_test = pl.from_pandas(df_test)    
    df_candidate = otto_common_fe.add_matrice_data_polars(df_test, buy2buy_matrice, df_candidate, col_name, n_max,
                                                          'order_predictions', normalize=True)
    return df_candidate

In [9]:
# Generates a feature using click2buy short co-visitation matrix. Short refers to time between click and buy events, only
# cases of relatively short period of time are counted.
# Uses no more than 15 events from the last hour of the session.
def add_click2buy_short_weight(input_path, c2buy_short_matrice, df_candidate):
    n_max = 15
    col_name = 'wgt_c2buy_short'
    time_frame = 60 * 60
    df_test = pd.read_parquet(input_path)
    df_test= otto_common.filter_by_time_and_n_max(df_test, time_frame, n_max)
    df_candidate = pl.from_pandas(df_candidate)
    df_test = pl.from_pandas(df_test)
    df_candidate = otto_common_fe.add_matrice_data_polars(df_test, c2buy_short_matrice, df_candidate, col_name, n_max,
                                                         'order_predictions')
    return df_candidate


In [10]:
# Generates a feature using click2buy "long" co-visitation matrix. 
# Uses no more than 30 events from the last 3 hours of each session.
def add_click2buy_weight(input_path, c2buy_matrice, df_candidate):
    n_max = 30
    col_name = 'wgt_c2buy_full'
    time_frame = 3 * 60 * 60
    df_test = pd.read_parquet(input_path)
    df_test= otto_common.filter_by_time_and_n_max(df_test, time_frame, n_max)
    df_candidate = pl.from_pandas(df_candidate)
    df_test = pl.from_pandas(df_test)
    df_candidate = otto_common_fe.add_matrice_data_polars(df_test, c2buy_matrice, df_candidate, col_name, n_max,
                                                          'order_predictions', normalize=True)
    return df_candidate

In [11]:
# Generates a feature using click2buy "long-visitation matrix. 
# Uses just 6 events from the last 3 hours of each session.
def add_click2buy_weight_6(input_path, c2buy_matrice, df_candidate):
    n_max = 6
    col_name = 'wgt_c2buy_6_from_full'
    time_frame = 3 * 60 * 60
    df_test = pd.read_parquet(input_path)    
    df_test= otto_common.filter_by_time_and_n_max(df_test, time_frame, n_max)
    df_candidate = pl.from_pandas(df_candidate)
    df_test = pl.from_pandas(df_test)
    df_candidate = otto_common_fe.add_matrice_data_polars(df_test, c2buy_matrice, df_candidate, col_name, n_max,
                                                         'order_predictions')
    return df_candidate

## Feature engineering for cross-validation datasets
For the orders model, two different cross-validation datasets are used to create the same features. Later two different models will be trained on those data and then their predictions will be averaged to get the final prediction.

In [12]:
# Input paths for the first cross-validation dataset.
# Most inputs are the same for both cross-validation datasets, but not all of them.
candidates_cv1 = '/kaggle/input/otto-validate-candidates-orders/candidates_order_cv1.parquet'
trunked_sessions = '/kaggle/input/otto-prepare-cv/cv_inputs.parquet'
buy2buy_matrice_cv = '/kaggle/input/otto-prepare-candidates-buys/matrix_buy2buy_cv.parquet'

click2buy_short_matrice_cv = '/kaggle/input/otto-click2buy-short/matrix_click2buy_cv_short.parquet'
click2buy_matrice_cv = '/kaggle/input/otto-prepare-candidates-buys/matrix_click2buy_cv.parquet'
conversions_cv = '/kaggle/input/create-counts-buys/conversions_cv.parquet'
all2order_conv_cv = '/kaggle/input/create-counts-buys/2order_conversions_cv.parquet'
click_before_buy_cv = '/kaggle/input/create-counts-buys/clicks_before_buy_cv.parquet'
median_time_viewed_cv = '/kaggle/input/create-counts-for-clicks/time_viewed_agg_cv.parquet'
history_path_cv = '/kaggle/input/create-counts-buys/history_change_cv1.parquet'
full_sessions = '/kaggle/input/otto-prepare-cv/cv_train.parquet'

counts_before_path = '/kaggle/input/create-counts-buys/daily_counts_orders_train.parquet'
counts_during_path = '/kaggle/input/create-counts-buys/daily_counts_orders_cv1.parquet'
counts_before_path_clicks = '/kaggle/input/create-counts-for-clicks/daily_counts_train.parquet'
counts_during_path_clicks = '/kaggle/input/create-counts-for-clicks/daily_counts_cv1.parquet'


In [13]:
%%time
# Imports candidates and builds features from co-visitation matrixes for the first cross-validation dataset.

df_cand = pd.read_parquet(candidates_cv1)
df_cand = otto_common_fe.cand_read_and_reduce(df_cand, 'order', True)

df_cand = add_buy2buy_weight(trunked_sessions, buy2buy_matrice_cv, df_cand)
df_cand = add_click2buy_short_weight(trunked_sessions, click2buy_short_matrice_cv, df_cand)
df_cand = add_click2buy_weight(trunked_sessions, click2buy_matrice_cv, df_cand)
df_cand = add_click2buy_weight_6(trunked_sessions, click2buy_matrice_cv, df_cand)

start normalizing
wgt_buy2buy
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
wgt_c2buy_short
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
start normalizing
wgt_c2buy_full
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
wgt_c2buy_6_from_full
0
1
2
3
4
5
CPU times: user 1h 5min 5s, sys: 17min 6s, total: 1h 22min 11s
Wall time: 34min 36s


In [14]:
%%time
# Builds all the other features. Takes significantly less time than building features from co-visitation matrixes.

#df_cand = pd.read_parquet(candidates_cv1)
#df_cand = otto_common.cand_read_and_reduce(df_cand, 'order', True)

df_cand = otto_common_fe.add_history_aid_features(trunked_sessions, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_history_agg_features(trunked_sessions, df_cand)
df_cand = otto_common_fe.add_time_viewed(trunked_sessions, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_type_last(trunked_sessions, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_daily_averages(counts_before_path_clicks,counts_during_path_clicks, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_daily_averages_same_day(counts_during_path_clicks, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_weekly_averages(counts_before_path,counts_during_path, df_cand, 'order_predictions')
df_cand = add_total_buys(full_sessions, df_cand)
df_cand = add_total_orders(full_sessions, df_cand)
df_cand = otto_common_fe.add_session_time(trunked_sessions, df_cand)
df_cand = otto_common_fe.count_events_3hours(trunked_sessions, df_cand)
df_cand = otto_common_fe.add_total_buys_in_session(trunked_sessions, df_cand)
df_cand = add_buys_in_session(trunked_sessions, df_cand)
df_cand = join_the_2order_conv(all2order_conv_cv, df_cand)
df_cand = otto_common_fe.add_clicks_before_buy(click_before_buy_cv, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_median_time_viewed(median_time_viewed_cv, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_history_similarity(history_path_cv, df_cand)

CPU times: user 3min 37s, sys: 55.1 s, total: 4min 32s
Wall time: 4min 23s


In [15]:
'''
%%time
# Was used to check all aids that were not selected during candidate generation.

#df_cand = pd.read_parquet(candidates_cv1)
df_cand = cand_read_and_reduce_test_not_cands(candidates_cv1, True)
df_cand = otto_common.add_history_aid_features(trunked_sessions, df_cand, 'order_predictions')
df_cand = otto_common.add_history_agg_features(trunked_sessions, df_cand)
df_cand = otto_common.add_time_viewed(trunked_sessions, df_cand, 'order_predictions')
df_cand = otto_common.add_type_last(trunked_sessions, df_cand, 'order_predictions')
df_cand = otto_common.add_daily_averages(counts_before_path_clicks,counts_during_path_clicks, df_cand, 'order_predictions')
df_cand = otto_common.add_weekly_averages(counts_before_path,counts_during_path, df_cand, 'order_predictions')
df_cand = add_total_buys(full_sessions, df_cand)
df_cand = add_total_orders(full_sessions, df_cand)
df_cand = otto_common.add_session_time(trunked_sessions, df_cand)
df_cand = otto_common.count_events_3hours(trunked_sessions, df_cand)
df_cand = otto_common.add_total_buys_in_session(trunked_sessions, df_cand)
df_cand = add_buys_in_session(trunked_sessions, df_cand)
df_cand = join_the_2order_conv(all2order_conv_cv, df_cand)
df_cand = otto_common.add_clicks_before_buy(click_before_buy_cv, df_cand, 'order_predictions')
df_cand = otto_common.add_median_time_viewed(median_time_viewed_cv, df_cand, 'order_predictions')
df_cand = otto_common.add_history_similarity(history_path_cv, df_cand)
'''

"\n%%time\n# Was used to check all aids that were not selected during candidate generation.\n\n#df_cand = pd.read_parquet(candidates_cv1)\ndf_cand = cand_read_and_reduce_test_not_cands(candidates_cv1, True)\ndf_cand = otto_common.add_history_aid_features(trunked_sessions, df_cand, 'order_predictions')\ndf_cand = otto_common.add_history_agg_features(trunked_sessions, df_cand)\ndf_cand = otto_common.add_time_viewed(trunked_sessions, df_cand, 'order_predictions')\ndf_cand = otto_common.add_type_last(trunked_sessions, df_cand, 'order_predictions')\ndf_cand = otto_common.add_daily_averages(counts_before_path_clicks,counts_during_path_clicks, df_cand, 'order_predictions')\ndf_cand = otto_common.add_weekly_averages(counts_before_path,counts_during_path, df_cand, 'order_predictions')\ndf_cand = add_total_buys(full_sessions, df_cand)\ndf_cand = add_total_orders(full_sessions, df_cand)\ndf_cand = otto_common.add_session_time(trunked_sessions, df_cand)\ndf_cand = otto_common.count_events_3hours(t

In [16]:
# Check resulting dataframe size and export to file.
size = df_cand.memory_usage(deep='True').sum()
print(naturalsize(size))
df_cand.to_parquet('cv_features_order.parquet')

862.0 MB


In [17]:
# Input paths for the second cross-validation dataset (only paths that are different between the cross-validation datasets).
candidates_cv2 = '/kaggle/input/otto-validate-candidates-orders/candidates_order_cv2.parquet'
trunked_sessions = '/kaggle/input/otto-prepare-cv/cv_inputs2.parquet'
history_path_cv2 = '/kaggle/input/create-counts-buys/history_change_cv2.parquet'

counts_during_path = '/kaggle/input/create-counts-buys/daily_counts_orders_cv2.parquet'
counts_during_path_clicks = '/kaggle/input/create-counts-for-clicks/daily_counts_cv2.parquet'

In [18]:
%%time
# Import candidates and build features from co-visitation matrixes for the second dataset.

df_cand = pd.read_parquet(candidates_cv2)
df_cand = otto_common_fe.cand_read_and_reduce(df_cand, 'order', True)
df_cand = add_buy2buy_weight(trunked_sessions, buy2buy_matrice_cv, df_cand)
df_cand = add_click2buy_short_weight(trunked_sessions, click2buy_short_matrice_cv, df_cand)
df_cand = add_click2buy_weight(trunked_sessions, click2buy_matrice_cv, df_cand)
df_cand = add_click2buy_weight_6(trunked_sessions, click2buy_matrice_cv, df_cand)

start normalizing
wgt_buy2buy
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
wgt_c2buy_short
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
start normalizing
wgt_c2buy_full
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
wgt_c2buy_6_from_full
0
1
2
3
4
5
CPU times: user 1h 9min 41s, sys: 17min 44s, total: 1h 27min 25s
Wall time: 34min 14s


In [19]:
%%time
# Build all the other features for the second dataset.

df_cand = otto_common_fe.add_history_aid_features(trunked_sessions, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_history_agg_features(trunked_sessions, df_cand)
df_cand = otto_common_fe.add_time_viewed(trunked_sessions, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_type_last(trunked_sessions, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_daily_averages(counts_before_path_clicks,counts_during_path_clicks, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_daily_averages_same_day(counts_during_path_clicks, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_weekly_averages(counts_before_path,counts_during_path, df_cand, 'order_predictions')
df_cand = add_total_buys(full_sessions, df_cand)
df_cand = add_total_orders(full_sessions, df_cand)
df_cand = otto_common_fe.add_session_time(trunked_sessions, df_cand)
df_cand = otto_common_fe.count_events_3hours(trunked_sessions, df_cand)
df_cand = otto_common_fe.add_total_buys_in_session(trunked_sessions, df_cand)
df_cand = add_buys_in_session(trunked_sessions, df_cand)
df_cand = join_the_2order_conv(all2order_conv_cv, df_cand)
df_cand = otto_common_fe.add_clicks_before_buy(click_before_buy_cv, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_median_time_viewed(median_time_viewed_cv, df_cand, 'order_predictions')
df_cand = otto_common_fe.add_history_similarity(history_path_cv2, df_cand)

CPU times: user 3min 55s, sys: 1min 19s, total: 5min 14s
Wall time: 4min 51s


In [20]:
# Check resulting dataframe size and export to file.
size = df_cand.memory_usage(deep='True').sum()
print(naturalsize(size))
df_cand.to_parquet('cv2_features_order.parquet')

860.5 MB


## Feature engineering for test dataset

In [21]:
# Input paths for test dataset.
candidates_test = '/kaggle/input/otto-validate-candidates-orders/candidates_order_test.parquet'
trunked_sessions = '/kaggle/input/otto-prepare-cv/test.parquet'
buy2buy_matrice_test = '/kaggle/input/otto-prepare-candidates-buys/matrix_buy2buy_test.parquet'

click2buy_short_matrice_test = '/kaggle/input/otto-click2buy-short/matrix_click2buy_test_short.parquet'
click2buy_matrice_test = '/kaggle/input/otto-prepare-candidates-buys/matrix_click2buy_test.parquet'
all2order_conv_test = '/kaggle/input/create-counts-buys/2order_conversions_test.parquet'
conversions_test = '/kaggle/input/create-counts-buys/conversions_test.parquet'
click_before_buy_test = '/kaggle/input/create-counts-buys/clicks_before_buy_test.parquet'
median_time_viewed_test = '/kaggle/input/create-counts-for-clicks/time_viewed_agg_test.parquet'
history_path_test = '/kaggle/input/create-counts-buys/history_change_test.parquet'
full_sessions = '/kaggle/input/otto-prepare-cv/train_full.parquet'

counts_before_path = '/kaggle/input/create-counts-buys/daily_counts_orders_test_full.parquet'
counts_during_path = '/kaggle/input/create-counts-buys/daily_counts_orders_test_trunked.parquet'
counts_before_path_clicks = '/kaggle/input/create-counts-for-clicks/daily_counts_test_full.parquet'
counts_during_path_clicks = '/kaggle/input/create-counts-for-clicks/daily_counts_test_trunked.parquet'

In [22]:
%%time
# Splits test dataset into 2 chunks and builds features for both chunks.
n_splits=2

for i in range(n_splits):
    df = pd.read_parquet(candidates_test)
    df_cand = otto_common.divide_df_by_column(df, n_splits, i, 'session')
    del df
    gc.collect()
    df_cand = otto_common_fe.cand_read_and_reduce(df_cand, 'order', False)
    df_cand = add_buy2buy_weight(trunked_sessions, buy2buy_matrice_test, df_cand)
    df_cand = add_click2buy_short_weight(trunked_sessions, click2buy_short_matrice_test, df_cand)
    df_cand = add_click2buy_weight(trunked_sessions, click2buy_matrice_test, df_cand)
    df_cand = add_click2buy_weight_6(trunked_sessions, click2buy_matrice_test, df_cand)
        
    df_cand = otto_common_fe.add_history_aid_features(trunked_sessions, df_cand, 'order_predictions')
    df_cand = otto_common_fe.add_history_agg_features(trunked_sessions, df_cand)
    df_cand = otto_common_fe.add_time_viewed(trunked_sessions, df_cand, 'order_predictions')
    df_cand = otto_common_fe.add_type_last(trunked_sessions, df_cand, 'order_predictions')
    df_cand = otto_common_fe.add_daily_averages(counts_before_path_clicks,counts_during_path_clicks, df_cand, 'order_predictions')
    df_cand = otto_common_fe.add_daily_averages_same_day(counts_during_path_clicks, df_cand, 'order_predictions')
    df_cand = otto_common_fe.add_weekly_averages(counts_before_path,counts_during_path, df_cand, 'order_predictions')
    df_cand = add_total_buys(full_sessions, df_cand)
    df_cand = add_total_orders(full_sessions, df_cand)
    df_cand = otto_common_fe.add_session_time(trunked_sessions, df_cand)
    df_cand = otto_common_fe.count_events_3hours(trunked_sessions, df_cand)
    df_cand = otto_common_fe.add_total_buys_in_session(trunked_sessions, df_cand)
    df_cand = add_buys_in_session(trunked_sessions, df_cand)
    df_cand = join_the_2order_conv(all2order_conv_test, df_cand)
    df_cand = otto_common_fe.add_clicks_before_buy(click_before_buy_test, df_cand, 'order_predictions')
    df_cand = otto_common_fe.add_median_time_viewed(median_time_viewed_test, df_cand, 'order_predictions')
    df_cand = otto_common_fe.add_history_similarity(history_path_test, df_cand)
    string_i = 'test_features_order_part_' + str(i) + '.parquet'
    df_cand.to_parquet(string_i)

start normalizing
wgt_buy2buy
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
wgt_c2buy_short
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
start normalizing
wgt_c2buy_full
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
wgt_c2buy_6_from_full
0
1
2
3
4
5
start normalizing
wgt_buy2buy
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
wgt_c2buy_short
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
start normalizing
wgt_c2buy_full
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
wgt_c2buy_6_from_full
0
1
2
3
4
5
CPU times: user 4h 23min 26s, sys: 1h 14min 27s, total: 5h 37min 54s
Wall time: 2h 46min 42s
